# Watson-Twitter Overview
this application takes a term search from twitter __tweets(q="@metia")__ and analysis the conversations of the feed of users who have discussed that topic.The output is the <a href="https://console.bluemix.net/docs/services/personality-insights/user-overview.html#overviewGuidelines">Personality Insights</a> 
#### Important Links and docs:
<a href="https://console.bluemix.net">Watson API docs</a><br>
<a href="https://www.ibm.com/watson/developercloud/personality-insights/api/v3/?cm_mc_uid=97628601784115056634132&cm_mc_sid_50200000=1505663413&cm_mc_sid_52640000=1505663607">API reference for Personality Insights</a><br>
<a href="https://dev.twitter.com/rest/public">twitter</a>

In [154]:
import twitter
import yaml
import pandas as pd
import numpy as np
import time

myKeys = yaml.load(open('/home/billmanh/Downloads/Key_file', 'r'))

consumer_key=myKeys['twitter']['APIkey']
consumer_secret=myKeys['twitter']['APIsecret']
access_token_key=myKeys['twitter']['accessToken']
access_token_secret=myKeys['twitter']['accessSecret']

## The Twitter Part

In [21]:
t = twitter.Twitter(
    auth=twitter.OAuth(access_token_key, access_token_secret, consumer_key, consumer_secret))

In [26]:
mentions = t.search.tweets(q="@metia")

In [159]:
df = pd.DataFrame()

In [160]:
df['text'] = [i['text'] for i in mentions['statuses']]
df['screen_name'] = [i['user']['screen_name'] for i in mentions['statuses']]
df['id'] = [i['user']['id'] for i in mentions['statuses']]
df['followers_count'] = [i['user']['followers_count'] for i in mentions['statuses']]
df['name'] = [i['user']['name'] for i in mentions['statuses']]
df['location'] = [i['user']['location'] for i in mentions['statuses']]

In [188]:
df

,text,screen_name,id,followers_count,name,location
0,Home again after fab week at #CMWorld Thanks t...,AbbeyCompton,1380319418,175,Abbey Compton,"Reston, Virginia"
1,RT @Metia: Listening to @misiatramp at #CMWorl...,andrewcm,16868831,816,Andrew Martin,Seattle
2,RT @Metia: Very pleased to be selected as a fi...,steveellis,9347102,1335,Steve Ellis,"Mostly London, often Seattle"
3,"RT @Metia: ""Empowering your sales team to clos...",steveellis,9347102,1335,Steve Ellis,"Mostly London, often Seattle"
4,RT @Metia: Listening to @misiatramp at #CMWorl...,MarciaRJohnston,71454459,3804,MarciaRieferJohnston,PDX
5,"""Anywhere there's a conversation is content."" ...",amykl888,3621989597,181,Amy Lyle,


In [195]:
def addFeed(screen_name):
    tempfd = pd.DataFrame()
    feed = t.statuses.user_timeline(screen_name=screen_name)
    tempfd['text'] = [i['text'] for i in feed]
    tempfd['screen_name'] = screen_name
    tempfd['created_at'] = [time.mktime(time.strptime(i['created_at'],"%a %b %d %H:%M:%S +0000 %Y")) for i in feed]
    tempfd['userID'] = df[df['screen_name']==screen_name]['id'].tolist()[0]
    tempfd['id'] = [i['id'] for i in feed]
    return pd.concat([fd,tempfd])

In [196]:
#fetch the feed of each person on in the previous query
fd = pd.DataFrame()

for i in np.unique(df['screen_name'].tolist()):
    fd = addFeed(i)
    
fd = fd.reset_index(drop=True)

In [197]:
fd.head()

,text,screen_name,created_at,userID,id
0,Home again after fab week at #CMWorld Thanks t...,AbbeyCompton,1.504980e+09,1380319418,906473767852924929
1,Thanks @hitRECordJoe for joining our #CMWorld ...,AbbeyCompton,1.504836e+09,1380319418,905867509403754497
2,@tamadear @Robert_Rose He likes you too! Calli...,AbbeyCompton,1.504757e+09,1380319418,905538214055858180
3,"RT @patriciajchui: Hello, Cleveland! Rock 'n' ...",AbbeyCompton,1.504757e+09,1380319418,905537439648931840
4,"Greetings, friends. Hope everyone had an amazi...",AbbeyCompton,1.504413e+09,1380319418,904093379180539904


In [220]:
#Example Conent Item
def formatContent(i):
    m = fd.ix[i]
    n = {
         "content": m.text,
         "contenttype": "text/plain",
         "created": int(m.created_at*1000), #watson time is in milliseconds,twitter time is in seconds
         "id": str(m.id),
         "language": "en"
      }
    return n

In [221]:
formatContent(2)

{'content': '@tamadear @Robert_Rose He likes you too! Calling out the red thread',
 'contenttype': 'text/plain',
 'created': 1504757308000,
 'id': '905538214055858180',
 'language': 'en'}

In [222]:
ind = fd[fd['screen_name']=='AbbeyCompton']

In [223]:
contentItems = [formatContent(i) for i in ind.index]

In [224]:
payload = {
   "contentItems": contentItems
    }

## The Watson part

In [215]:
import watson_developer_cloud

values = pd.DataFrame()
consumption_preferences = pd.DataFrame()
needs = pd.DataFrame()
personality = pd.DataFrame()
behavior = pd.DataFrame()

testAlias = 'AbbeyCompton'

personality_insights = watson_developer_cloud.PersonalityInsightsV3(
  version='2016-10-20',
  username=myKeys['watsonPersonalityInsights']['username'],
  password=myKeys['watsonPersonalityInsights']['password']
)

In [ ]:
values = pd.DataFrame()
consumption_preferences = pd.DataFrame()
needs = pd.DataFrame()
personality = pd.DataFrame()
behavior = pd.DataFrame()

In [226]:
profile = personality_insights.profile(
    payload, content_type='application/json',
    raw_scores=True, consumption_preferences=True)

In [228]:
profile.keys()

dict_keys(['consumption_preferences', 'values', 'word_count_message', 'warnings', 'needs', 'word_count', 'processed_language', 'personality', 'behavior'])

In [291]:
def getProfileFromWatson(payload):
    profile = personality_insights.profile(
    payload, content_type='application/json',
    raw_scores=True, consumption_preferences=True)
    return profile

def addProfileToDFs(profile,alias):
    #values
    for item in profile['values']:
        values.loc[alias,item['name']+' : percentile'] = item['percentile']
        values.loc[alias,item['name']+' : raw_score'] = item['raw_score']
    #consumption_preferences
    for item in profile['consumption_preferences']:
        pID = item['consumption_preference_category_id']
        for b in item['consumption_preferences']:
            consumption_preferences.loc[alias,b['name']+' : score'] = b['score']
    #needs
    for item in profile['needs']:
        needs.loc[alias,item['name']+' : percentile'] = item['percentile']
        needs.loc[alias,item['name']+' : raw_score'] = item['raw_score']
    #personality
    for item in profile['personality']:
        personality.loc[alias,item['name']+' : percentile'] = item['percentile']
        personality.loc[alias,item['name']+' : raw_score'] = item['raw_score']
    #behavior
    for item in profile['behavior']:
        behavior.loc[alias,item['name']+' : percentage'] = item['percentage']

In [292]:
values = pd.DataFrame()
consumption_preferences = pd.DataFrame()
needs = pd.DataFrame()
personality = pd.DataFrame()
behavior = pd.DataFrame()

for name in np.unique(df['screen_name'].tolist()):
    print(name)
    ind = fd[fd['screen_name']==name]
    contentItems = [formatContent(i) for i in ind.index]
    payload = {
       "contentItems": contentItems
    }
    profile = getProfileFromWatson(payload)
    addProfileToDFs(profile,name)

AbbeyCompton
MarciaRJohnston
amykl888
andrewcm
steveellis


In [293]:
values

,Conservation : percentile,Conservation : raw_score,Openness to change : percentile,Openness to change : raw_score,Hedonism : percentile,Hedonism : raw_score,Self-enhancement : percentile,Self-enhancement : raw_score,Self-transcendence : percentile,Self-transcendence : raw_score
AbbeyCompton,0.200826,0.627667,0.498548,0.784128,0.316857,0.707745,0.700191,0.729362,0.183461,0.818263
MarciaRJohnston,0.403063,0.654393,0.910212,0.836020,0.406129,0.723017,0.753866,0.736458,0.807131,0.847086
amykl888,0.705224,0.689753,0.580339,0.792087,0.268990,0.698839,0.277641,0.680452,0.783004,0.845702
andrewcm,0.509060,0.666473,0.508547,0.785094,0.279330,0.700822,0.462277,0.702176,0.413161,0.829385
steveellis,0.367103,0.650154,0.563969,0.790478,0.206312,0.685841,0.804365,0.743943,0.093304,0.811448


In [294]:
needs

,Challenge : percentile,Challenge : raw_score,Closeness : percentile,Closeness : raw_score,Curiosity : percentile,Curiosity : raw_score,Excitement : percentile,Excitement : raw_score,Harmony : percentile,Harmony : raw_score,...,Love : percentile,Love : raw_score,Practicality : percentile,Practicality : raw_score,Self-expression : percentile,Self-expression : raw_score,Stability : percentile,Stability : raw_score,Structure : percentile,Structure : raw_score
AbbeyCompton,0.779328,0.766314,0.301719,0.770304,0.389578,0.809270,0.306578,0.645044,0.452268,0.805116,...,0.691141,0.792938,0.692788,0.738658,0.265851,0.653465,0.240207,0.711977,0.584408,0.701855
MarciaRJohnston,0.903008,0.790037,0.654040,0.810949,0.940958,0.857783,0.584468,0.702151,0.927117,0.859487,...,0.857485,0.820666,0.866918,0.751797,0.886584,0.716730,0.761298,0.757076,0.724424,0.712229
amykl888,0.667076,0.751152,0.657307,0.811343,0.624797,0.825022,0.369552,0.658750,0.850205,0.845076,...,0.451966,0.762789,0.328166,0.718159,0.673478,0.690547,0.749814,0.755911,0.584696,0.701875
andrewcm,0.849647,0.778199,0.628105,0.807868,0.730038,0.832782,0.551204,0.695429,0.770102,0.834782,...,0.804823,0.810445,0.737051,0.741477,0.666237,0.689857,0.844178,0.766671,0.841274,0.723167
steveellis,0.890329,0.786871,0.232459,0.760925,0.213637,0.795756,0.490062,0.683226,0.343244,0.795320,...,0.480977,0.766339,0.760667,0.743080,0.279818,0.654915,0.346385,0.721866,0.634606,0.705402


In [295]:
personality

,Openness : percentile,Openness : raw_score,Conscientiousness : percentile,Conscientiousness : raw_score,Extraversion : percentile,Extraversion : raw_score,Agreeableness : percentile,Agreeableness : raw_score,Emotional range : percentile,Emotional range : raw_score
AbbeyCompton,0.649719,0.760930,0.640171,0.644791,0.884592,0.604477,0.642424,0.756707,0.803318,0.515481
MarciaRJohnston,0.688601,0.764356,0.445200,0.621624,0.845275,0.596361,0.743136,0.768160,0.386767,0.451145
amykl888,0.511002,0.749532,0.652037,0.646279,0.938828,0.619950,0.956264,0.810134,0.686355,0.494737
andrewcm,0.656962,0.761556,0.931867,0.697539,0.683171,0.572273,0.771901,0.771820,0.912262,0.543739
steveellis,0.592284,0.756104,0.776688,0.663547,0.532329,0.554626,0.359977,0.727946,0.773605,0.509688


In [296]:
consumption_preferences

,Likely to be sensitive to ownership cost when buying automobiles : score,Likely to prefer safety when buying automobiles : score,Likely to prefer quality when buying clothes : score,Likely to prefer style when buying clothes : score,Likely to prefer comfort when buying clothes : score,Likely to be influenced by brand name when making product purchases : score,Likely to be influenced by product utility when making product purchases : score,Likely to be influenced by online ads when making product purchases : score,Likely to be influenced by social media when making product purchases : score,Likely to be influenced by family when making product purchases : score,...,Likely to have experience playing music : score,Likely to like Latin music : score,Likely to like rock music : score,Likely to like classical music : score,Likely to read often : score,Likely to read entertainment magazines : score,Likely to read non-fiction books : score,Likely to read financial investment books : score,Likely to read autobiographical books : score,Likely to volunteer for social causes : score
AbbeyCompton,0.0,0.5,1.0,1.0,0.0,1.0,0.5,0.0,0.0,0.0,...,0.0,1.0,0.0,0.5,0.0,0.0,1.0,1.0,0.0,1.0
MarciaRJohnston,0.0,0.5,0.0,1.0,0.0,1.0,0.5,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
amykl888,0.0,1.0,0.0,1.0,0.0,0.5,0.5,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,1.0
andrewcm,0.0,0.5,1.0,1.0,0.0,1.0,0.5,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
steveellis,0.0,0.5,1.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
